In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, balanced_accuracy_score
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
# Import Diabetes_Prediction_Clean dataset into google colab
from google.colab import files
uploaded = files.upload()

Saving Diabetes_Prediction_Clean.csv to Diabetes_Prediction_Clean.csv


In [3]:
# Read the imported data
df_diabetes_data = pd.read_csv("Diabetes_Prediction_Clean.csv")
df_diabetes_data.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Male,28.0,0,0,never,27.32,5.7,158,0
2,Female,36.0,0,0,current,23.45,5.0,155,0
3,Male,76.0,1,1,current,20.14,4.8,155,0
4,Female,20.0,0,0,never,27.32,6.6,85,0


In [4]:
# Convert categorical data to numeric with " pd.get_dummies"
df_diabetes_clean = pd.get_dummies(df_diabetes_data)
df_diabetes_clean.head()

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes,gender_Female,gender_Male,gender_Other,smoking_history_current,smoking_history_former,smoking_history_never
0,80.0,0,1,25.19,6.6,140,0,1,0,0,0,0,1
1,28.0,0,0,27.32,5.7,158,0,0,1,0,0,0,1
2,36.0,0,0,23.45,5.0,155,0,1,0,0,1,0,0
3,76.0,1,1,20.14,4.8,155,0,0,1,0,1,0,0
4,20.0,0,0,27.32,6.6,85,0,1,0,0,0,0,1


In [5]:
df_diabetes_clean.shape

(60180, 13)

In [6]:
# Remove diabetes target from features data
y = df_diabetes_clean['diabetes'].values
X = df_diabetes_clean.drop(columns=['diabetes']).values



In [7]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [8]:
# Preprocess numerical data for neural network
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [9]:
# Check the number of columns to fed as features
len(X_train[0])


12

In [10]:
# Define the deep learning model
number_input_features = len(X_train[0])
hidden_layer1 = 20
hidden_layer2 = 10
hidden_layer3 = 5
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer1, activation='relu', input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation='tanh'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer3, activation='sigmoid'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                260       
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 5)                 55        
                                                                 
 dense_3 (Dense)             (None, 1)                 6         
                                                                 
Total params: 531
Trainable params: 531
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

In [12]:
# Train the model
fit_nn = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
1411/1411 [==============================] - 2s 1ms/step - loss: 0.1944 - accuracy: 0.9360
Epoch 2/200
1411/1411 [==============================] - 2s 1ms/step - loss: 0.1506 - accuracy: 0.9495
Epoch 3/200
1411/1411 [==============================] - 2s 1ms/step - loss: 0.1381 - accuracy: 0.9542
Epoch 4/200
1411/1411 [==============================] - 2s 1ms/step - loss: 0.1273 - accuracy: 0.9586
Epoch 5/200
1411/1411 [==============================] - 4s 3ms/step - loss: 0.1204 - accuracy: 0.9609
Epoch 6/200
1411/1411 [==============================] - 2s 2ms/step - loss: 0.1156 - accuracy: 0.9614
Epoch 7/200
1411/1411 [==============================] - 2s 1ms/step - loss: 0.1116 - accuracy: 0.9624
Epoch 8/200
1411/1411 [==============================] - 2s 1ms/step - loss: 0.1088 - accuracy: 0.9627
Epoch 9/200
1411/1411 [==============================] - 3s 2ms/step - loss: 0.1070 - accuracy: 0.9627
Epoch 10/200
1411/1411 [==============================] - 3s 2ms/step - l

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

471/471 - 0s - loss: 0.1062 - accuracy: 0.9638 - 379ms/epoch - 804us/step
Loss: 0.10622135549783707, Accuracy: 0.9637753367424011


In [25]:
# Create predictions variable
predictions = nn.predict(X_test_scaled)

471/471 [==============================] - 0s 966us/step


In [26]:
print((predictions>0.5).astype(int))

[[1]
 [0]
 [1]
 ...
 [0]
 [0]
 [0]]


In [27]:
predictions_thresholded = (predictions>0.5).astype(int)

In [29]:
# Evaluate model using a balanced accuracy score
balanced_accuracy = balanced_accuracy_score(y_test, predictions_thresholded)
print(f"Balanced Accuracy Score : {balanced_accuracy}")

Balanced Accuracy Score : 0.8411832078115971
